# Merging Turnstile Files to GeoLocation, Zip Codes, and Census


In [46]:
#Importing Libraries
import pandas as pd
import numpy as np
import pyspark.sql.functions as func
import datetime as dt
import re
import time
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import seaborn as sns

In [47]:
df_turnstile= pd.read_csv("Resources/Turnstile_data/Turnstile_Usage_Data__2016.csv", low_memory=False)
df_turnstile.head()

,C/A,Unit,SCP,Station,Line Name,Division,Date,Time,Description,Entries,Exits
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/30/2016,23:00:00,REGULAR,5991455,2028368
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/30/2016,19:00:00,REGULAR,5991003,2028335
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/30/2016,15:00:00,REGULAR,5989931,2028277
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/30/2016,07:00:00,REGULAR,5989153,2028014
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/30/2016,03:00:00,REGULAR,5989126,2027972


In [48]:
df_turnstile.Station.unique()

array(['59 ST', '5 AV/59 ST', '57 ST-7 AV', '49 ST', 'TIMES SQ-42 ST',
       '34 ST-HERALD SQ', '28 ST', '23 ST', '14 ST-UNION SQ', '8 ST-NYU',
       'PRINCE ST', 'CANAL ST', 'CITY HALL', 'CORTLANDT ST', 'RECTOR ST',
       'WHITEHALL S-FRY', 'DELANCEY/ESSEX', 'BOWERY', 'CHAMBERS ST',
       'FULTON ST', 'BROAD ST', '7 AV', 'PARK PLACE', 'BOTANIC GARDEN',
       'PROSPECT PARK', 'PARKSIDE AV', 'CHURCH AV', 'BEVERLEY ROAD',
       'CORTELYOU RD', 'NEWKIRK PLAZA', 'AVENUE H', 'AVENUE J',
       'AVENUE M', 'KINGS HWY', 'AVENUE U', 'NECK RD', 'SHEEPSHEAD BAY',
       'BRIGHTON BEACH', 'OCEAN PKWY', 'BOROUGH HALL', 'JAY ST-METROTEC',
       'DEKALB AV', 'ATL AV-BARCLAY', 'UNION ST', '4AV-9 ST',
       'PROSPECT AV', '25 ST', '36 ST', '45 ST', '53 ST', 'BAY RIDGE AV',
       '77 ST', '86 ST', 'BAY RIDGE-95 ST', '8 AV', 'FT HAMILTON PKY',
       'NEW UTRECHT AV', '18 AV', '20 AV', 'BAY PKWY', '9 AV', '50 ST',
       '55 ST', '71 ST', '79 ST', '25 AV', 'BAY 50 ST', 'CONEY IS-STILLW',
      

In [49]:
df_turnstile["Station"].nunique()

376

In [50]:
df_turnstile["Station"] = df_turnstile["Station"].str.lower()
df_turnstile.head()

,C/A,Unit,SCP,Station,Line Name,Division,Date,Time,Description,Entries,Exits
0,A002,R051,02-00-00,59 st,NQR456W,BMT,12/30/2016,23:00:00,REGULAR,5991455,2028368
1,A002,R051,02-00-00,59 st,NQR456W,BMT,12/30/2016,19:00:00,REGULAR,5991003,2028335
2,A002,R051,02-00-00,59 st,NQR456W,BMT,12/30/2016,15:00:00,REGULAR,5989931,2028277
3,A002,R051,02-00-00,59 st,NQR456W,BMT,12/30/2016,07:00:00,REGULAR,5989153,2028014
4,A002,R051,02-00-00,59 st,NQR456W,BMT,12/30/2016,03:00:00,REGULAR,5989126,2027972


In [51]:
df_geolocation = pd.read_csv("Resources/stops.csv", low_memory=False)
df_geolocation.head(10)

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
0,101,NaN,Van Cortlandt Park - 242 St,NaN,40.889248,-73.898583,NaN,NaN,1,NaN
1,101N,NaN,Van Cortlandt Park - 242 St,NaN,40.889248,-73.898583,NaN,NaN,0,101
2,101S,NaN,Van Cortlandt Park - 242 St,NaN,40.889248,-73.898583,NaN,NaN,0,101
3,103,NaN,238 St,NaN,40.884667,-73.900870,NaN,NaN,1,NaN
4,103N,NaN,238 St,NaN,40.884667,-73.900870,NaN,NaN,0,103
5,103S,NaN,238 St,NaN,40.884667,-73.900870,NaN,NaN,0,103
6,104,NaN,231 St,NaN,40.878856,-73.904834,NaN,NaN,1,NaN
7,104N,NaN,231 St,NaN,40.878856,-73.904834,NaN,NaN,0,104
8,104S,NaN,231 St,NaN,40.878856,-73.904834,NaN,NaN,0,104
9,106,NaN,Marble Hill - 225 St,NaN,40.874561,-73.909831,NaN,NaN,1,NaN


In [52]:
list(df_geolocation)

['stop_id',
 'stop_code',
 'stop_name',
 'stop_desc',
 'stop_lat',
 'stop_lon',
 'zone_id',
 'stop_url',
 'location_type',
 'parent_station']

In [53]:
df_geolocation["stop_name"].nunique()

380

In [54]:
new_df_geolocation = df_geolocation[['stop_name', 'stop_lat', 'stop_lon']].copy()
new_df_geolocation.head()

,stop_name,stop_lat,stop_lon
0,Van Cortlandt Park - 242 St,40.889248,-73.898583
1,Van Cortlandt Park - 242 St,40.889248,-73.898583
2,Van Cortlandt Park - 242 St,40.889248,-73.898583
3,238 St,40.884667,-73.900870
4,238 St,40.884667,-73.900870


In [55]:
new_df_geolocation["stop_name"] = new_df_geolocation["stop_name"].str.lower()
new_df_geolocation.head()

,stop_name,stop_lat,stop_lon
0,van cortlandt park - 242 st,40.889248,-73.898583
1,van cortlandt park - 242 st,40.889248,-73.898583
2,van cortlandt park - 242 st,40.889248,-73.898583
3,238 st,40.884667,-73.900870
4,238 st,40.884667,-73.900870


In [56]:
#Renaming columns Station Name to Station
new_df_geolocation.rename(columns = {'stop_name': 'Station','stop_lat': 'Station_lat', 'stop_lon': 'Station_lon'}, inplace=True)
new_df_geolocation.head()

,Station,Station_lat,Station_lon
0,van cortlandt park - 242 st,40.889248,-73.898583
1,van cortlandt park - 242 st,40.889248,-73.898583
2,van cortlandt park - 242 st,40.889248,-73.898583
3,238 st,40.884667,-73.900870
4,238 st,40.884667,-73.900870


In [57]:
new_df_geolocation.Station.unique()

array(['van cortlandt park - 242 st', '238 st', '231 st',
       'marble hill - 225 st', '215 st', '207 st', 'dyckman st', '191 st',
       '181 st', '168 st - washington hts', '157 st', '145 st',
       '137 st - city college', '125 st', '116 st - columbia university',
       'cathedral pkwy', '103 st', '96 st', '86 st', '79 st', '72 st',
       '66 st - lincoln center', '59 st - columbus circle', '50 st',
       'times sq - 42 st', '34 st - penn station', '28 st', '23 st',
       '18 st', '14 st', 'christopher st - sheridan sq', 'houston st',
       'canal st', 'franklin st', 'chambers st', 'cortlandt st',
       'rector st', 'south ferry loop', 'south ferry',
       'wakefield - 241 st', 'nereid av', '233 st', '225 st', '219 st',
       'gun hill rd', 'burke av', 'allerton av', 'pelham pkwy',
       'bronx park east', 'e 180 st', 'west farms sq - e tremont av',
       '174 st', 'freeman st', 'simpson st', 'intervale av',
       'prospect av', 'jackson av', '3 av - 149 st',
       '1

In [58]:
merged_turnstile_geo = pd.merge(df_turnstile, new_df_geolocation, on='Station', how='inner')
merged_turnstile_geo

,C/A,Unit,SCP,Station,Line Name,Division,Date,Time,Description,Entries,Exits,Station_lat,Station_lon
0,A002,R051,02-00-00,59 st,NQR456W,BMT,12/30/2016,23:00:00,REGULAR,5991455,2028368,40.762526,-73.967967
1,A002,R051,02-00-00,59 st,NQR456W,BMT,12/30/2016,23:00:00,REGULAR,5991455,2028368,40.762526,-73.967967
2,A002,R051,02-00-00,59 st,NQR456W,BMT,12/30/2016,23:00:00,REGULAR,5991455,2028368,40.762526,-73.967967
3,A002,R051,02-00-00,59 st,NQR456W,BMT,12/30/2016,23:00:00,REGULAR,5991455,2028368,40.641362,-74.017881
4,A002,R051,02-00-00,59 st,NQR456W,BMT,12/30/2016,23:00:00,REGULAR,5991455,2028368,40.641362,-74.017881
5,A002,R051,02-00-00,59 st,NQR456W,BMT,12/30/2016,23:00:00,REGULAR,5991455,2028368,40.641362,-74.017881
6,A002,R051,02-00-00,59 st,NQR456W,BMT,12/30/2016,19:00:00,REGULAR,5991003,2028335,40.762526,-73.967967
7,A002,R051,02-00-00,59 st,NQR456W,BMT,12/30/2016,19:00:00,REGULAR,5991003,2028335,40.762526,-73.967967
8,A002,R051,02-00-00,59 st,NQR456W,BMT,12/30/2016,19:00:00,REGULAR,5991003,2028335,40.762526,-73.967967
9,A002,R051,02-00-00,59 st,NQR456W,BMT,12/30/2016,19:00:00,REGULAR,5991003,2028335,40.641362,-74.017881


In [59]:
merged_turnstile_geo["Station"].nunique()

239

# Reverse Geocoding to find Zip Codes of Subway stations

In [60]:
df_zicode= pd.read_csv("Resources/geo_to_zipcode.csv", low_memory=False)
df_zicode.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,...,Accuracy Score,Accuracy Type,Number,Street,City,State,County,Zip,Country,Source
0,101,NaN,Van Cortlandt Park - 242 St,NaN,40.889248,-73.898583,NaN,NaN,1,NaN,...,1.0,rooftop,5959,Broadway,Bronx,NY,Bronx County,10463,US,City of New York
1,101N,NaN,Van Cortlandt Park - 242 St,NaN,40.889248,-73.898583,NaN,NaN,0,101,...,1.0,rooftop,5959,Broadway,Bronx,NY,Bronx County,10463,US,City of New York
2,101S,NaN,Van Cortlandt Park - 242 St,NaN,40.889248,-73.898583,NaN,NaN,0,101,...,1.0,rooftop,5959,Broadway,Bronx,NY,Bronx County,10463,US,City of New York
3,103,NaN,238 St,NaN,40.884667,-73.900870,NaN,NaN,1,NaN,...,1.0,rooftop,193,W 237th St,Bronx,NY,Bronx County,10463,US,City of New York
4,103N,NaN,238 St,NaN,40.884667,-73.900870,NaN,NaN,0,103,...,1.0,rooftop,193,W 237th St,Bronx,NY,Bronx County,10463,US,City of New York


In [61]:
list(df_zicode)

['stop_id',
 'stop_code',
 'stop_name',
 'stop_desc',
 'stop_lat',
 'stop_lon',
 'zone_id',
 'stop_url',
 'location_type',
 'parent_station',
 'Latitude',
 'Longitude',
 'Accuracy Score',
 'Accuracy Type',
 'Number',
 'Street',
 'City',
 'State',
 'County',
 'Zip',
 'Country',
 'Source']

In [62]:
new_df_zicode = df_zicode[['stop_lat', 'stop_lon', 'City', 'Zip', 'County' ]].copy()
new_df_zicode.head()


,stop_lat,stop_lon,City,Zip,County
0,40.889248,-73.898583,Bronx,10463,Bronx County
1,40.889248,-73.898583,Bronx,10463,Bronx County
2,40.889248,-73.898583,Bronx,10463,Bronx County
3,40.884667,-73.900870,Bronx,10463,Bronx County
4,40.884667,-73.900870,Bronx,10463,Bronx County


In [63]:
new_df_zicode.rename(columns = {'stop_lat': 'Station_lat', 'stop_lon': 'Station_lon'}, inplace=True)
new_df_zicode.head()

,Station_lat,Station_lon,City,Zip,County
0,40.889248,-73.898583,Bronx,10463,Bronx County
1,40.889248,-73.898583,Bronx,10463,Bronx County
2,40.889248,-73.898583,Bronx,10463,Bronx County
3,40.884667,-73.900870,Bronx,10463,Bronx County
4,40.884667,-73.900870,Bronx,10463,Bronx County


In [64]:
merged_turnstile_geo_zipcodes = pd.merge(merged_turnstile_geo, new_df_zicode, on=['Station_lat', 'Station_lon'], how='inner')
merged_turnstile_geo_zipcodes

,C/A,Unit,SCP,Station,Line Name,Division,Date,Time,Description,Entries,Exits,Station_lat,Station_lon,City,Zip,County
0,A002,R051,02-00-00,59 st,NQR456W,BMT,12/30/2016,23:00:00,REGULAR,5991455,2028368,40.762526,-73.967967,New York,10022,New York County
1,A002,R051,02-00-00,59 st,NQR456W,BMT,12/30/2016,23:00:00,REGULAR,5991455,2028368,40.762526,-73.967967,New York,10022,New York County
2,A002,R051,02-00-00,59 st,NQR456W,BMT,12/30/2016,23:00:00,REGULAR,5991455,2028368,40.762526,-73.967967,New York,10022,New York County
3,A002,R051,02-00-00,59 st,NQR456W,BMT,12/30/2016,23:00:00,REGULAR,5991455,2028368,40.762526,-73.967967,New York,10022,New York County
4,A002,R051,02-00-00,59 st,NQR456W,BMT,12/30/2016,23:00:00,REGULAR,5991455,2028368,40.762526,-73.967967,New York,10022,New York County
5,A002,R051,02-00-00,59 st,NQR456W,BMT,12/30/2016,23:00:00,REGULAR,5991455,2028368,40.762526,-73.967967,New York,10022,New York County
6,A002,R051,02-00-00,59 st,NQR456W,BMT,12/30/2016,23:00:00,REGULAR,5991455,2028368,40.762526,-73.967967,New York,10022,New York County
7,A002,R051,02-00-00,59 st,NQR456W,BMT,12/30/2016,23:00:00,REGULAR,5991455,2028368,40.762526,-73.967967,New York,10022,New York County
8,A002,R051,02-00-00,59 st,NQR456W,BMT,12/30/2016,23:00:00,REGULAR,5991455,2028368,40.762526,-73.967967,New York,10022,New York County
9,A002,R051,02-00-00,59 st,NQR456W,BMT,12/30/2016,19:00:00,REGULAR,5991003,2028335,40.762526,-73.967967,New York,10022,New York County


In [65]:
list(merged_turnstile_geo_zipcodes)

['C/A',
 'Unit',
 'SCP',
 'Station',
 'Line Name',
 'Division',
 'Date',
 'Time',
 'Description',
 'Entries',
 'Exits                                                     ',
 'Station_lat',
 'Station_lon',
 'City',
 'Zip',
 'County']

In [66]:
#merged_turnstile_geo_zipcodes.to_csv('merged_turnstile_geo_zipcodes.csv')

# Merging the census data with merged_turnstile_geo_zipcodes



In [67]:
df_census= pd.read_csv("Resources/clean_census_Output.csv", low_memory=False)
df_census

,Unnamed: 0,Year,ZIPCODE,Total Annual Payroll,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Whites Alone,Non Whites
0,0,2012,10001,8118354,20579.0,35.7,76138.0,74212.0,2672.0,12.9841100150639,13654.0,6925
1,1,2012,10002,585960,80323.0,39.9,32265.0,25518.0,22136.0,27.55873162107989,24852.0,55471
2,2,2012,10003,5244723,56614.0,32.2,91767.0,73196.0,6153.0,10.868336453880666,43932.0,12682
3,3,2012,10004,9670296,2780.0,33.8,127281.0,127628.0,300.0,10.79136690647482,2166.0,614
4,4,2012,10005,5947321,5724.0,28.4,120729.0,99403.0,590.0,10.307477288609364,4238.0,1486
5,5,2012,10006,2130252,2696.0,31.5,115787.0,98790.0,257.0,9.532640949554896,1871.0,825
6,6,2012,10007,2299621,6188.0,34.5,210125.0,156554.0,146.0,2.3594053005817712,4463.0,1725
7,7,2012,10009,378053,62397.0,37.0,59034.0,44705.0,12380.0,19.84069746942962,37680.0,24717
8,8,2012,10010,8044200,29396.0,35.3,95556.0,84619.0,3799.0,12.923527010477615,22061.0,7335
9,9,2012,10011,4399264,51853.0,40.3,101515.0,113300.0,4461.0,8.603166644167166,40428.0,11425


In [68]:
list(df_census)

['Unnamed: 0',
 'Year',
 'ZIPCODE',
 'Total Annual Payroll',
 'Population',
 'Median Age',
 'Household Income',
 'Per Capita Income',
 'Poverty Count',
 'Poverty Rate',
 'Whites Alone',
 'Non Whites']

In [69]:
df_census = df_census.drop('Unnamed: 0', 1)

In [70]:
df_census

,Year,ZIPCODE,Total Annual Payroll,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Whites Alone,Non Whites
0,2012,10001,8118354,20579.0,35.7,76138.0,74212.0,2672.0,12.9841100150639,13654.0,6925
1,2012,10002,585960,80323.0,39.9,32265.0,25518.0,22136.0,27.55873162107989,24852.0,55471
2,2012,10003,5244723,56614.0,32.2,91767.0,73196.0,6153.0,10.868336453880666,43932.0,12682
3,2012,10004,9670296,2780.0,33.8,127281.0,127628.0,300.0,10.79136690647482,2166.0,614
4,2012,10005,5947321,5724.0,28.4,120729.0,99403.0,590.0,10.307477288609364,4238.0,1486
5,2012,10006,2130252,2696.0,31.5,115787.0,98790.0,257.0,9.532640949554896,1871.0,825
6,2012,10007,2299621,6188.0,34.5,210125.0,156554.0,146.0,2.3594053005817712,4463.0,1725
7,2012,10009,378053,62397.0,37.0,59034.0,44705.0,12380.0,19.84069746942962,37680.0,24717
8,2012,10010,8044200,29396.0,35.3,95556.0,84619.0,3799.0,12.923527010477615,22061.0,7335
9,2012,10011,4399264,51853.0,40.3,101515.0,113300.0,4461.0,8.603166644167166,40428.0,11425


In [71]:
# df_census['Median Age'] = pd.to_numeric(df_census['Median Age'], errors='coerce')
# df_census['Household Income'] = pd.to_numeric(df_census['Household Income'], errors='coerce')
# df_census['Per Capita Income'] = pd.to_numeric(df_census['Per Capita Income'], errors='coerce')
cols = ['Total Annual Payroll',
 'Population',
 'Median Age',
 'Household Income',
 'Per Capita Income',
 'Poverty Count',
 'Poverty Rate',
 'Whites Alone',
 'Non Whites']
df_census[cols] = df_census[cols].apply(pd.to_numeric, errors='coerce', axis=1)

In [72]:
# cols = ['Median Age','Household Income','Per Capita Income']
df_census[cols] = df_census[df_census[cols] > 0.0][cols]
new_df_census = df_census.dropna()


In [73]:
new_df_census

,Year,ZIPCODE,Total Annual Payroll,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Whites Alone,Non Whites
0,2012,10001,8118354.0,20579.0,35.7,76138.0,74212.0,2672.0,12.984110,13654.0,6925.0
1,2012,10002,585960.0,80323.0,39.9,32265.0,25518.0,22136.0,27.558732,24852.0,55471.0
2,2012,10003,5244723.0,56614.0,32.2,91767.0,73196.0,6153.0,10.868336,43932.0,12682.0
3,2012,10004,9670296.0,2780.0,33.8,127281.0,127628.0,300.0,10.791367,2166.0,614.0
4,2012,10005,5947321.0,5724.0,28.4,120729.0,99403.0,590.0,10.307477,4238.0,1486.0
5,2012,10006,2130252.0,2696.0,31.5,115787.0,98790.0,257.0,9.532641,1871.0,825.0
6,2012,10007,2299621.0,6188.0,34.5,210125.0,156554.0,146.0,2.359405,4463.0,1725.0
7,2012,10009,378053.0,62397.0,37.0,59034.0,44705.0,12380.0,19.840697,37680.0,24717.0
8,2012,10010,8044200.0,29396.0,35.3,95556.0,84619.0,3799.0,12.923527,22061.0,7335.0
9,2012,10011,4399264.0,51853.0,40.3,101515.0,113300.0,4461.0,8.603167,40428.0,11425.0


In [74]:
#new_df_census.to_csv('cleaned_census_withou_negatives.csv')


In [75]:
list(new_df_census)

['Year',
 'ZIPCODE',
 'Total Annual Payroll',
 'Population',
 'Median Age',
 'Household Income',
 'Per Capita Income',
 'Poverty Count',
 'Poverty Rate',
 'Whites Alone',
 'Non Whites']

In [76]:
new_df_census = new_df_census.rename(columns = {'ZIPCODE': 'Zip'})
new_df_census.head()

,Year,Zip,Total Annual Payroll,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Whites Alone,Non Whites
0,2012,10001,8118354.0,20579.0,35.7,76138.0,74212.0,2672.0,12.984110,13654.0,6925.0
1,2012,10002,585960.0,80323.0,39.9,32265.0,25518.0,22136.0,27.558732,24852.0,55471.0
2,2012,10003,5244723.0,56614.0,32.2,91767.0,73196.0,6153.0,10.868336,43932.0,12682.0
3,2012,10004,9670296.0,2780.0,33.8,127281.0,127628.0,300.0,10.791367,2166.0,614.0
4,2012,10005,5947321.0,5724.0,28.4,120729.0,99403.0,590.0,10.307477,4238.0,1486.0


In [77]:
df_census_filtered = new_df_census[(new_df_census.Year == 2016)]
df_census_filtered.head()

,Year,Zip,Total Annual Payroll,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Whites Alone,Non Whites
2090,2016,10001,8118354.0,23332.0,35.8,85168.0,86347.0,3922.0,16.809532,14814.0,8518.0
2091,2016,10001,8717306.0,23332.0,35.8,85168.0,86347.0,3922.0,16.809532,14814.0,8518.0
2092,2016,10001,8907912.0,23332.0,35.8,85168.0,86347.0,3922.0,16.809532,14814.0,8518.0
2093,2016,10001,9226142.0,23332.0,35.8,85168.0,86347.0,3922.0,16.809532,14814.0,8518.0
2094,2016,10001,9003675.0,23332.0,35.8,85168.0,86347.0,3922.0,16.809532,14814.0,8518.0


In [37]:
#df_census_filtered.to_csv("df_census_filtered.csv")

In [45]:
df_census_filtered = df_census_filtered.groupby ("Zip")


AttributeError: Cannot access callable attribute 'groupby' of 'DataFrameGroupBy' objects, try using the 'apply' method

In [44]:
merged_turnstile_geo_zipcodes = merged_turnstile_geo_zipcodes.groupby("Zip")

AttributeError: Cannot access callable attribute 'groupby' of 'DataFrameGroupBy' objects, try using the 'apply' method

In [ ]:
merged_turnstile_geo_zipcodes_census_2016= pd.merge(merged_turnstile_geo_zipcodes, df_census_filtered, on='Zip', how='outer')
merged_turnstile_geo_zipcodes_census_2016

In [ ]:
merged_turnstile_geo_zipcodes_census_2016= pd.concat([merged_turnstile_geo_zipcodes, df_census_filtered],axis=1)